# Семинар 9: Практическое применение KNN и SVM

**Цель семинара:** На практике, шаг за шагом, разобрать полный цикл решения задачи бинарной классификации с помощью методов K-ближайших соседей (KNN) и Опорных векторов (SVM). Мы будем использовать библиотеку `scikit-learn`.

**Задача:** Классификация подводных объектов по данным сонара. Нам нужно определить, является ли объект **миной (`M`)** или **камнем (`R`)** на основе 60 сигналов, полученных с сонара на разных частотах.

**Почему этот датасет?**
*   Это сложная задача бинарной классификации.
*   Признаков много (60), и мы не можем просто визуализировать данные, чтобы "увидеть" решение.
*   Данные не являются линейно разделимыми, что позволит нам увидеть мощь нелинейных ядер в SVM.

### Шаг 1: Загрузка и первичный анализ данных

**Что делаем:** Загружаем данные из файла и смотрим на их структуру. Убеждаемся, что нет пропусков, и понимаем, какого типа наши признаки и цель.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Загружаем данные. У этого файла нет заголовков, поэтому header=None.
# Последний, 61-й столбец (индекс 60) - это наша цель.
df = pd.read_csv('https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/data/sonar.csv', header=None)

# Посмотрим на первые 5 строк
print("Первые 5 строк датасета:")
display(df.head())

# Посмотрим на общую информацию и проверим пропуски
print("\nИнформация о датасете:")
df.info()

# Посмотрим на баланс классов
print("\nБаланс классов (M - мина, R - камень):")
print(df[60].value_counts())

**Выводы из первого шага:**
*   Данные успешно загружены. У нас 208 объектов и 61 столбец.
*   Все 60 признаков имеют числовой тип (`float64`).
*   Целевая переменная (столбец 60) имеет тип `object` (строка).
*   Пропусков в данных нет.
*   Классы достаточно сбалансированы (111 Мин и 97 Камней), что хорошо для обучения.

### Шаг 2: Подготовка данных для обучения

**Что делаем:** Разделяем наши данные на матрицу признаков `X` и вектор целевой переменной `y`. Затем разбиваем их на обучающую и тестовую выборки. Это **ключевой** шаг для объективной оценки модели.

In [ ]:
from sklearn.model_selection import train_test_split

# X - это все столбцы, кроме последнего (признаки)
X = df.drop(60, axis=1)

# y - это последний столбец (цель)
y = df[60]

# Разделяем данные на обучающую (75%) и тестовую (25%) выборки
# random_state=42 гарантирует, что разделение будет всегда одинаковым, 
# что позволяет воспроизводить результаты.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

### Шаг 3: Создание и обучение модели KNN

**Что делаем:** Мы будем использовать `Pipeline` и `GridSearchCV`. 
*   `Pipeline` — это конвейер, который позволяет объединить несколько шагов обработки в один. В нашем случае это будут 'масштабирование данных' -> 'модель KNN'. Это очень удобно и предотвращает утечку данных из тестовой выборки.
*   `GridSearchCV` — инструмент для автоматического подбора лучших гиперпараметров. Мы зададим ему диапазон значений `k` (числа соседей), а он найдет оптимальное.

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# Шаг 3.1: Создаем конвейер (Pipeline)
# Первый шаг - 'scaler': стандартизация данных (приведение к среднему 0 и дисперсии 1)
# Второй шаг - 'knn': наша модель
knn_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier())
])

# Шаг 3.2: Определяем сетку параметров для поиска
# Мы хотим, чтобы GridSearchCV перебрал все k от 1 до 29.
# Название параметра 'knn__n_neighbors' состоит из имени шага в Pipeline ('knn'), 
# двух подчеркиваний '__' и названия параметра в модели ('n_neighbors').
param_grid_knn = {'knn__n_neighbors': range(1, 30)}

# Шаг 3.3: Создаем и обучаем GridSearchCV
# cv=5 означает 5-кратную кросс-валидацию для оценки каждого параметра.
grid_knn = GridSearchCV(knn_pipe, param_grid_knn, cv=5, scoring='accuracy')

# Запускаем процесс обучения и подбора параметров
grid_knn.fit(X_train, y_train)

# Выводим лучший найденный параметр
print(f"Лучшее значение K для KNN: {grid_knn.best_params_['knn__n_neighbors']}")

### Шаг 4: Оценка качества модели KNN

**Что делаем:** Используя лучшую модель, найденную `GridSearchCV`, делаем предсказания на тестовых данных и оцениваем их качество с помощью матрицы ошибок и отчета о классификации.

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Делаем предсказания на тестовой выборке
y_pred_knn = grid_knn.predict(X_test)

print("\n--- Отчет по качеству модели KNN ---")
print(classification_report(y_test, y_pred_knn))

print("Матрица ошибок для KNN:")
print(confusion_matrix(y_test, y_pred_knn))

**Интерпретация результатов KNN:**
*   `classification_report` показывает нам `precision`, `recall` и `f1-score` для каждого класса.
*   `accuracy` (общая точность) показывает долю правильных ответов.
*   Матрица ошибок показывает, какие классы модель путает. В данном случае, модель с k=1 ошиблась на 5 камнях (предсказала их как мины) и на 1 мине (предсказала как камень).

### Шаг 5: Создание, обучение и оценка модели SVM

**Что делаем:** Повторяем тот же процесс, но теперь для модели SVM. Здесь мы будем подбирать уже другие гиперпараметры: `C` (штраф за ошибку) и `gamma` (влияние одного примера).

In [ ]:
from sklearn.svm import SVC

# Шаг 5.1: Создаем Pipeline для SVM
svm_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVC())
])

# Шаг 5.2: Определяем сетку параметров для SVM
# Мы переберем комбинации C и gamma. Ядро оставим 'rbf' - самое универсальное.
param_grid_svm = {
    'svm__C': [0.1, 1, 10, 50, 100],
    'svm__gamma': ['scale', 'auto', 0.1, 0.01, 0.001]
}

# Шаг 5.3: Создаем и обучаем GridSearchCV для SVM
grid_svm = GridSearchCV(svm_pipe, param_grid_svm, cv=5, scoring='accuracy')
grid_svm.fit(X_train, y_train)

# Выводим лучшие параметры
print(f"Лучшие параметры для SVM: {grid_svm.best_params_}")

# Шаг 5.4: Оценка лучшей модели SVM
y_pred_svm = grid_svm.predict(X_test)
print("\n--- Отчет по качеству модели SVM ---")
print(classification_report(y_test, y_pred_svm))

print("Матрица ошибок для SVM:")
print(confusion_matrix(y_test, y_pred_svm))

### Шаг 6: Итоговое сравнение моделей

**Что делаем:** Сводим результаты в одну таблицу или просто сравниваем отчеты, чтобы сделать вывод о том, какая модель лучше справилась с данной задачей.

**Результаты:**

| Метрика | KNN (k=1) | SVM (C=10, gamma='scale') |
| :--- | :--- | :--- |
| **Accuracy** | 0.90 | **0.94** |
| **F1-score (M)** | 0.92 | **0.95** |
| **F1-score (R)** | 0.89 | **0.93** |

**Общий вывод семинара:**

1.  Мы успешно реализовали полный цикл ML для двух разных моделей: от загрузки данных до оценки качества.
2.  Мы увидели, насколько важен подбор гиперпараметров и как `GridSearchCV` автоматизирует этот процесс.
3.  Мы убедились в необходимости использования `Pipeline` для корректного масштабирования данных.
4.  В данной конкретной задаче, после подбора гиперпараметров, **модель SVM показала себя немного лучше**, чем KNN, по всем ключевым метрикам. Это часто бывает в задачах, где граница между классами сложная и нелинейная.